### Stylesheet

In [5]:
%%html

<style>
img[alt="_"]{
    max-width: 500px;
    display: inline-block;
}
</style>

### Packages

In [6]:
import time
import asyncio
import requests
from multiprocessing import Process
import os
from Crypto.Hash import SHA256
from base64 import b64encode, b64decode
from multiprocessing import Pool
from multiprocessing import Process, Queue

try:
    import websockets
except:
    !pip install websockets

---

# Python with Async

### Note flow

1. CPU-BOUND vs I/O-BOUND

2. 파이썬-스레드-GIL

3. 동기와 비동기

4. 비동기와 멀티 프로세스

5. 비동기 웹페이지 크롤링

# 두 종류의 Task

- CPU Bound

> - CPU 자원을 사용하는 Task입니다.
> - 속도가 빠릅니다.
> - 파이썬 프로세스의 스레드에서 처리됩니다.
> - javascript로 치면 v8엔진에서 처리되는 작업입니다.

- I/O Bound

> - Disk, network, Database의 Input/Output과 관련된 Task입니다.

> 작업 환경 별 비용

> ![_](img/3.png)
> [출처](http://www.nextree.co.kr/p7292/)

# 쓰레드와 GIL

### 모두가 아는 사실

- 파이썬 프로그램은 기본적으로 하나의 쓰레드(Single Thread)에서 실행됩니다.

- 멀티 쓰레드에서 동작하도록 프로그래밍 해도, 전역 인터프리터 락킹(Global Interpreter Lock) 때문에 특정 시점에 하나의 파이썬 코드만을 실행하게 됩니다. 

### 어쩌면 새로운 사실

- 하지만 사실 이는 파이썬만의 특징이 아닙니다. 자바스크립트 등을 포함한 대부분의 인터프리터 언어는 [여러 문제를 방지하기 위해](https://www.linux.co.kr/home2/board/bbs/board.php?bo_table=lecture&wr_id=1642&sca=1&sca2=32) 이처럼 싱글쓰레드로 동작하도록 설계되었습니다. 다만 이러한 구조를 언급하는 용어나 문화 등이 다를 뿐입니다.

### GIL

1. [출처](https://yinjae.wordpress.com/2012/04/02/python-thread/)

> Lock의 detail함에 따라서 두 종류의 개념으로 설명을 하는데 coarse-grained-lock과 fine-grained-lock 이다. 말 그대로 coarse-grained-lock은 거친/굵은 개념의 lock으로 그냥 대충 크게 묶어서 lock을 설정하는것이고 fine-grained-lock은 반대로 lock을 아주 세밀히 나눠서 lock을 설정하는 개념이다. 위와 같은 개념으로 lock을 나눈다고 하였을 때, GIL은 coarse-grained-lock의 극단을 추구한 개념이다.

2. [출처](https://yinjae.wordpress.com/2012/04/02/python-thread/)

> - fine-grained-lock에 비해서 single thread일때 훨씬 빠르다.
> - fine-grained-lock에 비해서 i/o bound program에 한해서 multi-thread 인 경우 더 빠를 수 있다.
> - GIL은 blocking i/o call의 경우 해제되므로 성능에 영향을 미치지 않는다.
> - fine-grained-lock에 비해서 C library를 사용하는 연산이 많은(compute-intensive) cpu bound program을 multi-thread로 돌리는 경우 더 빠를 수 있다.
> - C와 포트란의 연산라이브러리를 사용할 때, GIL은 해제되므로 성능에 영향을 미치지 않는다.
> - C extension을 쓰기에 용이하다.
> - C extension에 의해서 GIL은 해제된다.
> - C library의 wrapping이 용이하다.
> - C library의 thread-safety를 고려하지 않아도 된다. GIL을 이용해서 lock을 잡고 돌리면 thread-safety에 상관없이 정상적으로 동작하게 된다.
> - ### 따라서 결과적으로 python으로만 짜여진 CPU bound program의 경우에만 thread가 비효율적이라고 한다. 

3.

> - 이는 표준으로 쓰고있는 CPython에 국한되는 이야기입니다. JPython(JAVA)이나 IronPython(C#)에서는 멀티쓰레드 병렬연산이 가능합니다.

# 비동기 방식과 I/O-Bound Task

- GIL

![_](img/6.png)
[출처](http://highthroughput.org/wp/cb-1136/)

- 멀티 쓰레드

![_](img/4.png)
[출처](http://www.nextree.co.kr/p7292/)

- 비동기 방식

![_](img/5.png)
[출처](http://www.nextree.co.kr/p7292/)

> 파이썬(또는 JavaScript)에서 I/O-Bound 작업에 대해서는 GIL이 잠시 해제됩니다. 이를 응용해 I/O-Bound 작업은 굳이 멀티쓰레드 방식이 아니더라도 병렬작업이 가능합니다.

> ### 이경우 비동기 방식이 유용합니다.

# 비동기 방식에 대한 간단한 설명

> 사실 구글링으로 비동기 공부를 하고 싶다면, 파이썬 보다는 자바스크립트를 기반으로 찾아보는게 훨씬 수월합니다.

> 대부분의 파이썬 사용자들은 동기화 방식의 프로그래밍에 익숙하지만,

> 자바스크립트는 웹의 상호작용을 위해 만들어 졌기 때문에 태생부터 비동기적 환경에서 발전한 언어입니다.

> 다른 언어지만 비동기 프로그래밍 방식은 모두가 비슷합니다. (사실 여기 첨부된 사진 중 절반이, 그 출처가 nodejs 블로그입니다.)

![_](img/9.png)
[출처](https://opentutorials.org/course/2136/11884)

![_](img/7.png)
[출처](http://www.nextree.co.kr/p7292/)

## 비동기 프로그래밍은 아래의 방법들로 이루어집니다

### 1. 콜백
![_](img/8.png)
[출처](http://www.nextree.co.kr/p7292/)

### 2. Promise
![_](img/10.png)
[출처](https://blogs.msdn.microsoft.com/windowsappdev/2013/06/11/all-about-promises-for-windows-store-apps-written-in-javascript/)

# 예제코드를 통한 더 자세한 설명
# Synchronous

In [57]:
def is_prime(x):
    
    return not any(x // i == x / i for i in range(2, x - 1))


def highest_prime_below(x):
    
    print("run function for %d" % x)
    
    for y in range(x-1, 1, -1):
        if is_prime(y):
            print("Highest prime below %d is %d" % (x, y))
            break
        time.sleep(0.05)
    else:
        print("%d is smallest prime" % x)
    return None


def main():
    
    start = time.time()
    highest_prime_below(100000)
    highest_prime_below(10000)
    highest_prime_below(1000)
    highest_prime_below(2)
    print(time.time() - start)
    
    
if __name__ == "__main__":
    main()

run function for 100000
Highest prime below 100000 is 99991
run function for 10000
Highest prime below 10000 is 9973
run function for 1000
Highest prime below 1000 is 997
run function for 2
2 is smallest prime
1.919736623764038


# Asynchronous

In [58]:
async def highest_prime_below(x):

    print("run function for %d" % x)
    
    for y in range(x-1, 1, -1):
        if is_prime(y):
            print("Highest prime below %d is %d" % (x, y))
            break
        await asyncio.sleep(0.05)
    else:
        print("%d is smallest prime" % x)
    return None


async def main():
    
    start = time.time()
    
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000),
        highest_prime_below(2)])
    
    print(time.time() - start)
    

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

run function for 10000
run function for 1000
run function for 2
2 is smallest prime
run function for 100000
Highest prime below 1000 is 997
Highest prime below 100000 is 99991
Highest prime below 10000 is 9973
1.3685705661773682


# CPU-BOUND EXAMPLE

In [59]:
N = 100000000
D = 5

def function(from_, to):
    
    print(sum(range(from_, to)))

# 단순처리

In [60]:
start = time.time()
dvs = [[N // D * i, N // D * (i + 1)] for i in range(D)]
for dv in dvs:
    function(*dv)
print(time.time() - start)

199999990000000
599999990000000
999999990000000
1399999990000000
1799999990000000
1.7600266933441162


# 비동기처리

In [62]:
async def each_get(from_, to):

    await loop.run_in_executor(None, function, from_, to)

    
async def coget():
    
    dvs = [[N // D * i, N // D * (i + 1)] for i in range(D)]
    await asyncio.gather(*[each_get(*dv) for dv in dvs])
    
start = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(coget())
record = time.time() - start
time.sleep(2)
print(record)

599999990000000999999990000000
17999999900000001999999900000001399999990000000



1.6208219528198242


# I/O-BOUND EXAMPLE

### Network case

In [63]:
N = 4

def FUNCTION(start):
    requests.get("https://sites.google.com/a/chromium.org/chromedriver/getting-started")
    print(time.time() - start)

### Disk case

In [40]:
N = 4
MB = 500

with open("./js/data", "wb") as f:
    f.write(b"\x00" * 1000 * 1000 * MB)

def FUNCTION(start):
    with open("./js/data") as f:
        print(time.time() - start)

# 단순처리

In [53]:
if __name__ == "__main__":

    start = time.time()
    for _ in range(N):
        FUNCTION(start)

1.750410795211792
1.9228851795196533
2.1076250076293945
4.762524366378784


# 비동기처리

In [56]:
async def each_get():

    await loop.run_in_executor(None, FUNCTION, start)

    
async def coget():
    
    await asyncio.wait([each_get() for _ in range(N)])
    

if __name__ == "__main__":

    start = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(coget())

1.0566723346710205
1.06704831123352051.0675597190856934

1.1320886611938477


# 비동기 프로그래밍이 산업에서 이용되는 구체적 예시
# Chatting Server

In [ ]:
async def handler(websocket, path):

    global connected
    connected.add(websocket)
    n_of_con = len(connected)
    
    while True:
        await asyncio.sleep(2)
        if len(connected) == n_of_con:
            for ws in connected:
                try:
                    message = str(len(connected))
                    await ws.send(message)
                except:
                    pass
            

if __name__ == "__main__":

    connected = set()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        websockets.serve(handler, '0.0.0.0', 5678))
    loop.run_forever()

# 비동기 프로그래밍을 알고 난 후 확인해보는 멀티프로세스
# 윈도우에서는 주피터(REPL)가 아닌 스크립트로 돌려야 합니다!
![](img/11.png)

# 프로세스란

- 하나의 CPU 즉 프로세서는 한 순간에 하나의 프로세스만 실행할 수 있습니다. (CPU 코어당 한개씩)

- 하지만 트릭으로 여러개 프로세스를 동시에 구동합니다. ([이부분은 운영체제 전문가에게 문의하세요.](http://bowbowbow.tistory.com/16))

- 따라서 진정한(순수한) 병렬성으로 얻을 수 있는 이득은 {(코어 개수) * (1 - 부가적비용)} 만큼입니다.

![_](img/1.png)

In [66]:
!ps

  PID TTY          TIME CMD
   71 pts/1    00:00:00 sh
   72 pts/1    00:00:00 ps


In [67]:
def GO(N, D):

    list(range(N // D))
    print("parent process: {} | process id: {}".format(os.getppid(), os.getpid()))
    
    
if __name__=='__main__':

    N = 100000000
    D = os.cpu_count()
    
    start = time.time()

    processes = []

    for i in range(D):
        proc = Process(target=GO, args=(N, D))
        processes.append(proc)
    
    for proc in processes:
        proc.start()
    
    for proc in processes:
        proc.join()
        
    print(time.time() - start)

parent process: 20 | process id: 73
parent process: 20 | process id: 75
parent process: 20 | process id: 74
parent process: 20 | process id: 76
1.1049268245697021


In [68]:
!ps

  PID TTY          TIME CMD
   85 pts/1    00:00:00 sh
   86 pts/1    00:00:00 ps


# 메모리를 공유하는 하나의 풀에서 멀티 프로세스를 수행할 수 있습니다

# Pool

In [ ]:
def f(x):
    
    print("process id: {} | value: {}".format(os.getpid(), x))
    return os.getpid()


if __name__ == '__main__':
        
    D = os.cpu_count()
    
    with Pool(D) as p:
        print(set(p.map(f, range(8))))

# 큐를 만들어 그 큐를 통해 프로세스끼리 자원을 공유할 수 있습니다.

# Queue

In [14]:
def f1(q):
    
    data = list(range(10))
    q.put(data)

    
def f2(q):
    
    data = list(range(10, 20))
    data = q.put(data)

    
    
if __name__ == '__main__':
    
    q = Queue()
    
    p1 = Process(target=f1, args=(q,))
    p2 = Process(target=f2, args=(q,))
    p1.start()
    p2.start()

    p1.join()
    p2.join()
    
    print(q.get())
    print(q.get())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


# 이 영상 추천합니다!
# https://vimeo.com/96425312

![](img/12.png)

# 효과적인 웹 크롤링을 위한 꿀팁들
# 1. 암호화

In [14]:
if __name__ == "__main__":

    hashing = SHA256.new()
    hashing.update(b'seohasong')

    암호화된값 = hashing.hexdigest()
    쿠키값 = b64encode(암호화된값.encode("utf-8")).decode("utf-8")
    쿠키값디코딩 = b64decode(쿠키값).decode("utf-8")
    
    print("암호화된값: ", 암호화된값)
    print()
    print("쿠키값: ", 쿠키값)
    print()
    print("쿠키값디코딩: ", 쿠키값디코딩)

암호화된값:  29de565b5a62403b46790bac6efeb40a8f1f2161b03cd23b2fbdeeb96abd72d4

쿠키값:  MjlkZTU2NWI1YTYyNDAzYjQ2NzkwYmFjNmVmZWI0MGE4ZjFmMjE2MWIwM2NkMjNiMmZiZGVlYjk2YWJkNzJkNA==

쿠키값디코딩:  29de565b5a62403b46790bac6efeb40a8f1f2161b03cd23b2fbdeeb96abd72d4


# 2. 세션 탈취

![_](img/2.png)

In [15]:
cookie = "remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6Imd4bkNSYUUrbDBsbTdyOVFTRTBCdkE9PSIsInZhbHVlIjoiUmFvZ0ZLdmlWTGcrMzgxQ0U5d3lrdDFPRU1BS0h6dXUrMXlrb0pMdWZlWmdNXC9CaTJtOWtQZW1ka0ZxZ0ZNNHhPTjFZMzVNSnlOQVo2MTI1RWNleWxhUjRCaElhU1BJbmlyK1IraU1tajZVPSIsIm1hYyI6IjVjYjNiOTFhNTdkODE1ZjJiMTkwYzdlZGFkMGNjMjExMGUwNWJjMTg0OWZhZDExMzU2YTE1ZmFkMzNiMzY3NDAifQ"

headers = {
    "Cookie": cookie
}

json_data = requests.get("https://api.klue.kr/info/mypage", headers=headers).text
쿠키값디코딩 = b64decode(cookie.split("=")[-1] + "==").decode("utf-8")

print("json_data: ", json_data)
print()
print("쿠키값디코딩: ", 쿠키값디코딩)

json_data:  {"code":200,"data":{"id":"tisutoo","sc_id":"ku","sc_email":"tisutoo","phone_number":null,"coffee_bean":0,"points":10,"evaluation_count":24,"note_count":0,"purchase_note_count":0,"facebook":null,"google":null,"read_lec_eval_authority":true}}

쿠키값디코딩:  {"iv":"gxnCRaE+l0lm7r9QSE0BvA==","value":"RaogFKviVLg+381CE9wykt1OEMAKHzuu+1ykoJLufeZgM\/Bi2m9kPemdkFqgFM4xON1Y35MJyNAZ6125EceylaR4BhIaSPInir+R+iMmj6U=","mac":"5cb3b91a57d815f2b190c7edad0cc2110e05bc1849fad11356a15fad33b36740"}


# 3. Headless Chrome

[고려대학교 강의평가 사이트는 SPA로 만들어져 있습니다.](https://klue.kr/) 이 사이트를 다음과 같은 간단한 코드로 크롤링할 수 있습니다.

```bash
# alias chrome="/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome"
chrome --headless --disable-gpu --dump-dom https://klue.kr/ > index.html
(sleep 5; rm -rf index.html) & (chrome http://localhost:1111; python -m SimpleHTTPServer 1111)
;
```

# 세션 탈취 + Selenium Webdriver + Headless Chrome == 무적의 크롤러

- https://sites.google.com/a/chromium.org/chromedriver/getting-started